In [1]:
### instructions:
# 1.
# * copy-paste ipythonwidget.py

# 2. for debug:
# * comment self._init_static()

# 3. for release check:
# * uncomment self._init_static()
# * replace
#    return os.path.join(os.path.dirname(__file__), file_name)
#   with
#    return os.path.join(os.path.dirname('__your_local_repo_dir__/catboost/python-package/catboost/widget/'), file_name)

import os
import csv
import time
import json
from threading import Thread
from IPython.core.display import display, HTML
from traitlets import Unicode, Dict, default
from ipywidgets import DOMWidget, Layout, widget_serialization


class CatboostIpythonWidget(DOMWidget):
    _view_name = Unicode('CatboostIpythonWidgetView').tag(sync=True)
    _view_module = Unicode('catboost_module').tag(sync=True)

    data = Dict({}).tag(sync=True, **widget_serialization)

    def __init__(self, train_dir):
        super(self.__class__, self).__init__()
        self._train_dir = train_dir

    @default('layout')
    def _default_layout(self):
        return Layout(height='500px', align_self='stretch')

    def update_widget(self, subdirs=False):
        # wait for start train (meta.tsv)
        # self._init_static()
        time.sleep(1.0)
        self._update_data(subdirs=subdirs)

        display(self)

        while self._need_update:
            self._update_data(subdirs=subdirs)
            time.sleep(2.0)

    def _run_update(self):
        thread = Thread(target=self.update_widget, args=())
        thread.start()

    def _get_subdirectories(self, a_dir):
        return [{'name': name, 'path': os.path.join(a_dir, name)}
            for name in os.listdir(a_dir) if os.path.isdir(os.path.join(a_dir, name))]

    def _update_data(self, subdirs=False):
        data = {}
        dirs = [{'name': 'current', 'path': self._train_dir}]
        need_update = False

        if subdirs:
            dirs = self._get_subdirectories(self._train_dir)

        for dir_info in dirs:
            path = dir_info.get('path')
            content = self._update_data_from_dir(path)

            if not content:
                continue

            data[path] = {
                'path': path,
                'name': dir_info.get('name'),
                'content': content
            }

            if not need_update:
                need_update = data[path]['content']['passed_iterations'] + 1 < data[path]['content']['total_iterations']

        self.data = data
        self._need_update = need_update

    def _update_data_from_dir(self, path):
        data = {
            'iterations': {},
            'meta': {}
        }
        
        training_json = os.path.join(path, 'catboost_training.json')
        
        if os.path.isfile(training_json):
             with open(training_json, 'r') as json_data:
                training_data = json.load(json_data)
                
                data['meta'] = training_data['meta']
                data['iterations'] = training_data['iterations']
                   
        else:
            return None

        return {
            'passed_iterations': data['iterations'][-1]['iteration'],
            'total_iterations': data['meta']['iteration_count'],
            'data': data
        }

    @staticmethod
    def _get_static_path(file_name):
        return os.path.join(os.path.dirname('/Users/ivan-karev/github/catboost/catboost/python-package/catboost/widget/'), file_name)

    def _init_static(self):
        with open(self._get_static_path('CatboostIpython.css')) as f:
            css = f.read()
        js = ''

        # never use require in your projects
        js += 'window.__define = window.define;window.__require = window.require;window.define = undefined;window.require = undefined;'
        with open(self._get_static_path('plotly-basic.min.js')) as f:
            js += f.read()
        js += 'window.define = window.__define;window.require = window.__require;window.__define = undefined; window.__require = undefined;'

        with open(self._get_static_path('CatboostIpythonPlotly.js')) as f:
            js += f.read()
        with open(self._get_static_path('CatboostIpythonInit.js')) as f:
            js += f.read()
        html = """
            <style>
                {}
            </style>
            <script>
                {}
            </script>
        """.format(css, js)

        display(HTML(html))

In [2]:
%%javascript

// http://stackoverflow.com/questions/32170197/how-do-can-i-use-a-custom-js-file-under-jupyter-notebook

// instructions:
// 1.
// copy-paste CatboostIpythonInit.js

// 2.
// replace
//  var debug = false;
// with
//  var debug = true;

// 3. setup local notebook:
/*
mkdir ~/.jupyter/custom
cd ~/.jupyter/custom
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/CatboostIpythonPlotly.js CatboostIpythonPlotly.js
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/CatboostIpython.css CatboostIpython.css
ln -s __your_local_repo_dir__/catboost/python-package/catboost/widget/plotly-basic.min.js plotly-basic.min.js
*/


var debug = true;

if (debug) {
    require.config({
        shim:{
            "custom/CatboostIpythonPlotly":{
                deps:["custom/plotly-basic.min"]
            } 
        }
    })
    
    require.undef('catboost_module');
    require.undef('custom/CatboostIpythonPlotly');
}

var moduleBase = '@jupyter-widgets/base';
var modules = [moduleBase];

if (debug) {
    modules.push('custom/CatboostIpythonPlotly');
}

define('catboost_module', modules, function(widgets) {
    var getInstance = function(el) {
            var id = $(el).attr('catboost-id');

            if (!id) {
                return null;
            }

            id = id.replace('catboost_', '');

            if (!window.catboostIpythonInstances[id]) {
                return null;
            }

            return window.catboostIpythonInstances[id];
        },
        addInstance = function(el) {
            $(el).attr('catboost-id', 'catboost_' + window.catboostIpythonIndex);

            var catboostIpython = new CatboostIpython();
            catboostIpython.index = window.catboostIpythonIndex;
            catboostIpython.plotly = window.Plotly;
            if (debug) {
                catboostIpython.loadStyles('/custom/CatboostIpython.css', function(){catboostIpython.resizeCharts();})
            }

            window.catboostIpythonInstances[window.catboostIpythonIndex] = catboostIpython;

            window.catboostIpythonIndex++;

            return catboostIpython;
        };

    var CatboostIpythonWidget = widgets.DOMWidgetView.extend({

        initialize: function() {
            CatboostIpythonWidget.__super__.initialize.apply(this, arguments);

            if (!window.catboostIpythonInstances) {
                window.catboostIpythonInstances = {};
            }

            if (typeof window.catboostIpythonIndex === 'undefined') {
                window.catboostIpythonIndex = 0;
            }

            var catboostIpythonInstance = getInstance(this.el);

            if (!catboostIpythonInstance) {
                catboostIpythonInstance = addInstance(this.el);
            }

            catboostIpythonInstance.init();
        },

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },

        update: function() {
            this.value_changed();
        },

        value_changed: function() {
            this.el.style['width'] = this.model.get('width');
            this.el.style['height'] = this.model.get('height');
            this.displayed.then(_.bind(this.render_charts, this));
        },

        process_all: function(parent, params) {
            var data = params.data;

            for (var path in data) {
                if (data.hasOwnProperty(path)) {
                    this.process_row(parent, data[path])
                }
            }
        },

        process_row: function(parent, data) {
            var catboostIpython = getInstance(parent),
                path = data.path,
                content = data.content,
                items = content.data.iterations,
                firstIndex = 0,
                chunks = [];
            
            if (!items || !items.length) {
                return;
            }
            
            if (!catboostIpython.lastIndex) {
                catboostIpython.lastIndex = {} 
            }

            if (catboostIpython.lastIndex[path]) {
                firstIndex = catboostIpython.lastIndex[path] + 1;
            }

            catboostIpython.lastIndex[path] = items.length - 1;

            for (var i = firstIndex; i < items.length; i++) {
                chunks.push(items[i]);
            }
            
            catboostIpython.addMeta(data.path, content.data.meta);

            catboostIpython.addPoints(parent, {
                chunks: chunks,
                train: data.name,
                path: data.path
            });
        },

        render_charts: function () {
            this.process_all(this.el, {
                data: this.model.get('data')
            });

            return this;
        }
    });

    return {
        CatboostIpythonWidgetView: CatboostIpythonWidget
    };
});

<IPython.core.display.Javascript object>

In [3]:
w = CatboostIpythonWidget('new_cv_1')
w.update_widget()

CatboostIpythonWidget(data={'new_cv_1': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'Accuracy'}, {u'best_value': u'Max', u'name': u'AUC'}], u'learn_sets': [u'fold_0_learn', u'fold_1_learn', u'fold_2_learn'], u'launch_mode': u'CV', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'Accuracy'}, {u'best_value': u'Max', u'name': u'AUC'}], u'iteration_count': 200, u'test_sets': [u'fold_0_test', u'fold_1_test', u'fold_2_test']}, 'iterations': [{u'fold_0_learn': [0.664862785, 0.8419072482, 0.8591273834], u'iteration': 0, u'remaining_time': 54.11629666, u'passed_time': 0.2719411892, u'fold_2_learn': [0.6649214976, 0.8429558541, 0.8606378072], u'fold_1_learn': [0.6652787185, 0.8435316699, 0.8605718864], u'fold_0_test': [0.6651863448, 0.8390910487, 0.8567818933], u'fold_1_test': [0.6655930645, 0.8425982801, 0.860448961], u'fold_2_test': [0.6656640942, 0.84375, 0.8603745116]}, {u'fold_0_learn': [0.6389630902, 0.8418304668, 0.85921438], u'iteration': 1, u'remaining_time': 46.92264054, u'passed_time': 0.4739660661, u'fold_2_learn': [0.6390945851, 0.8425335893, 0.8628204367], u'fold_1_learn': [0.6393588659, 0.8435316699, 0.8630855773], u'fold_0_test': [0.6395736956, 0.8392445877, 0.8569063981], u'fold_1_test': [0.6401631284, 0.8419840295, 0.8601237126], u'fold_2_test': [0.6404818421, 0.843980344, 0.8607345149]}, {u'fold_0_learn': [0.6156280587, 0.8419072482, 0.8636182946], u'iteration': 2, u'remaining_time': 44.16869433, u'passed_time': 0.6726197105, u'fold_2_learn': [0.6152213457, 0.8447600768, 0.8641094735], u'fold_1_learn': [0.6164119636, 0.8447984645, 0.865468811], u'fold_0_test': [0.616277447, 0.8392445877, 0.8628911934], u'fold_1_test': [0.617693154, 0.8432125307, 0.862321613], u'fold_2_test': [0.6171789135, 0.8457463145, 0.8615960676]}, {u'fold_0_learn': [0.5938911862, 0.843865172, 0.8648709778], u'iteration': 3, u'remaining_time': 42.150639, u'passed_time': 0.8602171224, u'fold_2_learn': [0.594651572, 0.8447600768, 0.862948445], u'fold_1_learn': [0.5956519209, 0.843646833, 0.8638590431], u'fold_0_test': [0.5947276093, 0.8412405957, 0.864077575], u'fold_1_test': [0.5971366101, 0.8419840295, 0.8618648433], u'fold_2_test': [0.5968968105, 0.8457463145, 0.861595679]}, {u'fold_0_learn': [0.5736021283, 0.8439419533, 0.8652178751], u'iteration': 4, u'remaining_time': 41.16539378, u'passed_time': 1.055522918, u'fold_2_learn': [0.5753442419, 0.8442994242, 0.8746012902], u'fold_1_learn': [0.576375916, 0.8432629559, 0.8747012219], u'fold_0_test': [0.5747036096, 0.8412405957, 0.8643746641], u'fold_1_test': [0.5782666483, 0.8413697789, 0.8737600624], u'fold_2_test': [0.5781445533, 0.8452856265, 0.8725030065]}, {u'fold_0_learn': [0.555348573, 0.8428670147, 0.865269554], u'iteration': 5, u'remaining_time': 40.41171964, u'passed_time': 1.249846999, u'fold_2_learn': [0.558377975, 0.8423800384, 0.8745223552], u'fold_1_learn': [0.5598085487, 0.842571977, 0.8743227469], u'fold_0_test': [0.5565207009, 0.8398587441, 0.8644771042], u'fold_1_test': [0.5619218428, 0.8417536855, 0.8737656336], u'fold_2_test': [0.5615429766, 0.8449785012, 0.8728774883]}, {u'fold_0_learn': [0.5393396579, 0.8444410319, 0.8648288864], u'iteration': 6, u'remaining_time': 39.90062705, u'passed_time': 1.447173002, u'fold_2_learn': [0.543715737, 0.8434932821, 0.8743222611], u'fold_1_learn': [0.5451312653, 0.842840691, 0.874005497], u'fold_0_test': [0.5406525218, 0.8415476739, 0.864284511], u'fold_1_test': [0.5474089249, 0.8414465602, 0.8738918752], u'fold_2_test': [0.5471207943, 0.8455927518, 0.8726327949]}, {u'fold_0_learn': [0.5248095054, 0.844210688, 0.8651166146], u'iteration': 7, u'remaining_time': 39.31935637, u'passed_time': 1.638306515, u'fold_2_learn': [0.528660281, 0.8429942418, 0.8744207882], u'fold_1_learn': [0.5302684977, 0.8437236084, 0.8742146504], u'fold_0_test': [0.5262023776, 0.8414709043, 0.8642734544], u'fold

In [4]:
w = CatboostIpythonWidget('new_cv_2')
w.update_widget()

CatboostIpythonWidget(data={'new_cv_2': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'AUC'}], u'learn_sets': [u'fold_0_learn', u'fold_1_learn', u'fold_2_learn'], u'launch_mode': u'CV', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'AUC'}], u'iteration_count': 200, u'test_sets': [u'fold_0_test', u'fold_1_test', u'fold_2_test']}, 'iterations': [{u'fold_0_learn': [0.360031649, 0.8591273834], u'iteration': 0, u'remaining_time': 54.22820407, u'passed_time': 0.2725035381, u'fold_2_learn': [0.357894903, 0.8606378072], u'fold_1_learn': [0.3587206921, 0.8605718864], u'fold_0_test': [0.364410523, 0.8567818933], u'fold_1_test': [0.356645542, 0.860448961], u'fold_2_test': [0.3583022038, 0.8603745116]}, {u'fold_0_learn': [0.3193044189, 0.9028804653], u'iteration': 3, u'remaining_time': 43.55687705, u'passed_time': 0.8889158582, u'fold_2_learn': [0.3190489587, 0.9037435111], u'fold_1_learn': [0.3195802247, 0.9030534169], u'fold_0_test': [0.3279524045, 0.89758268], u'fold_1_test': [0.3185146261, 0.9028345762], u'fold_2_test': [0.3189128028, 0.904117674]}, {u'fold_0_learn': [0.311384506, 0.9084396449], u'iteration': 6, u'remaining_time': 41.95384592, u'passed_time': 1.52164208, u'fold_2_learn': [0.3104189365, 0.9082007678], u'fold_1_learn': [0.3119313946, 0.9072540336], u'fold_0_test': [0.3229453305, 0.9019581075], u'fold_1_test': [0.3103442977, 0.9070064944], u'fold_2_test': [0.3121839961, 0.9078360292]}, {u'fold_0_learn': [0.3041137717, 0.9138800731], u'iteration': 9, u'remaining_time': 40.68960645, u'passed_time': 2.141558234, u'fold_2_learn': [0.3060769611, 0.9111352106], u'fold_1_learn': [0.3052769697, 0.9119747106], u'fold_0_test': [0.3159182703, 0.9075597077], u'fold_1_test': [0.3045009292, 0.9114822672], u'fold_2_test': [0.3086356684, 0.9107843022]}, {u'fold_0_learn': [0.301165812, 0.9153691619], u'iteration': 12, u'remaining_time': 41.10852019, u'passed_time': 2.857811564, u'fold_2_learn': [0.3027180452, 0.9125602099], u'fold_1_learn': [0.301105897, 0.914700093], u'fold_0_test': [0.3118700637, 0.9093304736], u'fold_1_test': [0.3010991972, 0.9137425677], u'fold_2_test': [0.3061091366, 0.9118214758]}, {u'fold_0_learn': [0.2988947448, 0.9166303759], u'iteration': 15, u'remaining_time': 40.1827601, u'passed_time': 3.494153052, u'fold_2_learn': [0.296232614, 0.9169378758], u'fold_1_learn': [0.2988527024, 0.916231772], u'fold_0_test': [0.3106789175, 0.9102228094], u'fold_1_test': [0.298369263, 0.9159393992], u'fold_2_test': [0.3016091429, 0.9152184371]}, {u'fold_0_learn': [0.2937921282, 0.9195286098], u'iteration': 18, u'remaining_time': 39.38288005, u'passed_time': 4.134114481, u'fold_2_learn': [0.2918948674, 0.9193651552], u'fold_1_learn': [0.2970515, 0.9171927832], u'fold_0_test': [0.3053954699, 0.9132758892], u'fold_1_test': [0.2972218121, 0.916452674], u'fold_2_test': [0.2966036318, 0.9179336773]}, {u'fold_0_learn': [0.2925703768, 0.920202873], u'iteration': 21, u'remaining_time': 38.6525995, u'passed_time': 4.777287578, u'fold_2_learn': [0.2912238219, 0.919696415], u'fold_1_learn': [0.2914062796, 0.9204643635], u'fold_0_test': [0.3045593639, 0.9141753156], u'fold_1_test': [0.2915877297, 0.9199239551], u'fold_2_test': [0.2975204656, 0.9175575759]}, {u'fold_0_learn': [0.2912593511, 0.9209861502], u'iteration': 24, u'remaining_time': 37.93885824, u'passed_time': 5.419836891, u'fold_2_learn': [0.290224464, 0.9202745592], u'fold_1_learn': [0.2870064596, 0.9225974339], u'fold_0_test': [0.3043869992, 0.9142300485], u'fold_1_test': [0.2872369783, 0.9222732645], u'fold_2_test': [0.2981049266, 0.9173024204]}, {u'fold_0_learn': [0.2904900421, 0.9213302458], u'iteration': 27, u'remaining_time': 37.19473986, u'passed_time': 6.054957652, u'fold_2_learn': [0.288620174, 0.9213637569], u'fold_1_learn': [0.2850788314, 0.9236597654], u'fold_0_test': [0.3034992706, 0.914572546], u'fold_1_test': 

In [5]:
w = CatboostIpythonWidget('new_cv_3')
w.update_widget()

CatboostIpythonWidget(data={'new_cv_3': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'AUC'}, {u'best_value': u'Max', u'name': u'Accuracy'}], u'learn_sets': [u'fold_0_learn', u'fold_1_learn', u'fold_2_learn'], u'launch_mode': u'CV', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Max', u'name': u'AUC'}, {u'best_value': u'Max', u'name': u'Accuracy'}], u'iteration_count': 200, u'test_sets': [u'fold_0_test', u'fold_1_test', u'fold_2_test']}, 'iterations': [{u'fold_0_learn': [0.683536871, 0.8591273834, 0.8419072482], u'iteration': 0, u'remaining_time': 54.17797516, u'passed_time': 0.2722511315, u'fold_2_learn': [0.6835569446, 0.8606378072, 0.8429558541], u'fold_1_learn': [0.6836828305, 0.8605718864, 0.8435316699], u'fold_0_test': [0.6836470527, 0.8567818933, 0.8390910487], u'fold_1_test': [0.6837903336, 0.860448961, 0.8425982801], u'fold_2_test': [0.6838120194, 0.8603745116, 0.84375]}, {u'fold_0_learn': [0.6013694006, 0.8652019593, 0.844210688], u'iteration': 10, u'remaining_time': 38.19610572, u'passed_time': 2.223053772, u'fold_2_learn': [0.6022084691, 0.8754588481, 0.8430710173], u'fold_1_learn': [0.6029165893, 0.874990375, 0.8423800384], u'fold_0_test': [0.6022184548, 0.8642946773, 0.8413941348], u'fold_1_test': [0.6045692008, 0.8737403204, 0.8417536855], u'fold_2_test': [0.6045926159, 0.8725510415, 0.843289312]}, {u'fold_0_learn': [0.5402073755, 0.8802941086, 0.8445562039], u'iteration': 20, u'remaining_time': 35.38271978, u'passed_time': 4.151045337, u'fold_2_learn': [0.542357827, 0.8755665033, 0.8424952015], u'fold_1_learn': [0.5428540491, 0.8750522641, 0.8422264875], u'fold_0_test': [0.5418672744, 0.8800940823, 0.841701213], u'fold_1_test': [0.5454558143, 0.8740457945, 0.8414465602], u'fold_2_test': [0.5460812649, 0.8729275315, 0.8428286241]}, {u'fold_0_learn': [0.4956297265, 0.8834987274, 0.8448633292], u'iteration': 30, u'remaining_time': 34.05361578, u'passed_time': 6.246521237, u'fold_2_learn': [0.4995750438, 0.8746490398, 0.8416506718], u'fold_1_learn': [0.5006399337, 0.8769648387, 0.8413435701], u'fold_0_test': [0.4978280967, 0.8836650122, 0.841701213], u'fold_1_test': [0.5035234729, 0.8774063651, 0.8414465602], u'fold_2_test': [0.5038872284, 0.8729916321, 0.8428286241]}, {u'fold_0_learn': [0.465642317, 0.8930240813, 0.8451320639], u'iteration': 40, u'remaining_time': 33.92590978, u'passed_time': 8.748190571, u'fold_2_learn': [0.4688628854, 0.8848524524, 0.8440307102], u'fold_1_learn': [0.46957713, 0.8846660768, 0.8444913628], u'fold_0_test': [0.4681624185, 0.8911560229, 0.8420850606], u'fold_1_test': [0.4732598597, 0.8871768861, 0.8449017199], u'fold_2_test': [0.4739521026, 0.8854752393, 0.8454391892]}, {u'fold_0_learn': [0.4446343065, 0.8950376722, 0.8465909091], u'iteration': 50, u'remaining_time': 31.24061888, u'passed_time': 10.69309774, u'fold_2_learn': [0.4453939555, 0.8870360655, 0.8437619962], u'fold_1_learn': [0.4468028795, 0.8859064572, 0.8441074856], u'fold_0_test': [0.4474192774, 0.8931662311, 0.84385076], u'fold_1_test': [0.4511368563, 0.8898102109, 0.8449785012], u'fold_2_test': [0.4508293293, 0.889317909, 0.8462070025]}, {u'fold_0_learn': [0.4252033201, 0.8964994153, 0.8500076781], u'iteration': 60, u'remaining_time': 28.8656663, u'passed_time': 12.6676665, u'fold_2_learn': [0.425203569, 0.8892189905, 0.8457965451], u'fold_1_learn': [0.4279524382, 0.8877796291, 0.845681382], u'fold_0_test': [0.4282443288, 0.8945246789, 0.8491478581], u'fold_1_test': [0.4320639635, 0.891986102, 0.8468980344], u'fold_2_test': [0.4307480433, 0.8911623489, 0.8471283784]}, {u'fold_0_learn': [0.409579554, 0.8979019852, 0.8503531941], u'iteration': 70, u'remaining_time': 26.65337564, u'passed_time': 14.66968737, u'fold_2_learn': [0.4104943684, 0.890968947, 0.8469865643], u'fold_1_learn': [0.4135587229, 0.8899741758, 0.8470633397], u'fold_0_test': [0.4128258793, 0.8958327485, 0.84991555

In [6]:
w = CatboostIpythonWidget('new_train')
w.update_widget()

CatboostIpythonWidget(data={'new_train': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Min', u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'learn_sets': [u'learn'], u'launch_mode': u'Train', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': u'Min', u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [0.6600097926, 0.8186408447, 0.7882716049], u'passed_time': 0.01195065927, u'iteration': 0, u'remaining_time': 0.2270625262, u'learn': [0.6565050203, 0.8045194116, 0.9427502877]}, {u'test': [0.632232761, 0.7521424448, 0.8009259259], u'passed_time': 0.01321400713, u'iteration': 1, u'remaining_time': 0.1189260641, u'learn': [0.6255581951, 0.7309581064, 0.9462025316]}, {u'test': [0.6095533415, 0.6986701294, 0.8104938272], u'passed_time': 0.01747301169, u'iteration': 2, u'remaining_time': 0.0990137329, u'learn': [0.5972445019, 0.6674712405, 0.9571346375]}, {u'test': [0.5900602047, 0.6505211024, 0.8703703704], u'passed_time': 0.01988600819, u'iteration': 3, u'remaining_time': 0.07954403276, u'learn': [0.576111189, 0.6163792864, 0.9833141542]}, {u'test': [0.5665918765, 0.5981484265, 0.8734567901], u'passed_time': 0.02221930921, u'iteration': 4, u'remaining_time': 0.06665792764, u'learn': [0.5480418822, 0.551996609, 0.9907940161]}, {u'test': [0.5510878118, 0.5675146714, 0.8666666667], u'passed_time': 0.02381639127, u'iteration': 5, u'remaining_time': 0.05557157962, u'learn': [0.530200324, 0.5112019609, 0.9838895282]}, {u'test': [0.5371883975, 0.5458694973, 0.862345679], u'passed_time': 0.02759446774, u'iteration': 6, u'remaining_time': 0.05124686866, u'learn': [0.5093274822, 0.4693390275, 0.9815880322]}, {u'test': [0.5196093687, 0.5164592834, 0.8641975309], u'passed_time': 0.02926329191, u'iteration': 7, u'remaining_time': 0.04389493786, u'learn': [0.4892220617, 0.4324720121, 0.9867663982]}, {u'test': [0.514718047, 0.4997642919, 0.8641975309], u'passed_time': 0.03006744424, u'iteration': 8, u'remaining_time': 0.03674909852, u'learn': [0.4848357164, 0.4135370112, 0.9867663982]}, {u'test': [0.5070544057, 0.4767807491, 0.8648148148], u'passed_time': 0.03104546668, u'iteration': 9, u'remaining_time': 0.03104546668, u'learn': [0.476894303, 0.391844182, 0.9884925201]}, {u'test': [0.4884107139, 0.4656478835, 0.8691358025], u'passed_time': 0.03491730462, u'iteration': 10, u'remaining_time': 0.02856870378, u'learn': [0.4559441537, 0.368178344, 0.9942462601]}, {u'test': [0.475222362, 0.4649231047, 0.8691358025], u'passed_time': 0.03872574833, u'iteration': 11, u'remaining_time': 0.02581716555, u'learn': [0.4395892343, 0.3601795584, 0.9902186421]}, {u'test': [0.4671390682, 0.4702490774, 0.8635802469], u'passed_time': 0.03969476613, u'iteration': 12, u'remaining_time': 0.02137410484, u'learn': [0.4299335594, 0.3634165349, 0.9856156502]}, {u'test': [0.4593684635, 0.4640512426, 0.8709876543], u'passed_time': 0.04063294628, u'iteration': 13, u'remaining_time': 0.01741411983, u'learn': [0.4239420933, 0.3563966828, 0.9907940161]}, {u'test': [0.4580933529, 0.4878625883, 0.8546296296], u'passed_time': 0.04445887295, u'iteration': 14, u'remaining_time': 0.01481962432, u'learn': [0.4128026493, 0.3622160541, 0.9896432681]}, {u'test': [0.4485422823, 0.4976934801, 0.8614197531], u'passed_time': 0.04693425378, u'iteration': 15, u'remaining_time': 0.01173356344, u'learn': [0.4014032279, 0.3732812248, 0.9907940161]}, {u'test': [0.440040623, 0.52335625, 0.8682098765], u'passed_time': 0.05097190937, u'iteration': 16, u'remaining_time': 0.00899504283, u'learn': [0.3894202204, 0.4013746303, 0.9936708861]}, {u'test': [0.4323807262, 0.5508922729, 0.862037037], u'passed_time': 0.05450196203, u'iteration': 17, u'remaining_time': 0.006055773559, u'learn': [0.3762092106, 0.4249888747, 0.9919447641]}, {u'test': [0.426786483, 0.5779252281, 0.8601851852], u'

In [7]:
w = CatboostIpythonWidget('new_train_with_nan')
w.update_widget()

CatboostIpythonWidget(data={'new_train_with_nan': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'MAPE'}, {u'best_value': u'Min', u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'learn_sets': [u'learn'], u'launch_mode': u'Train', u'learn_metrics': [{u'best_value': u'Min', u'name': u'MAPE'}, {u'best_value': u'Min', u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'iteration_count': 20, u'test_sets': [u'test']}, 'iterations': [{u'test': [u'inf', u'inf', 0.5], u'passed_time': 0.0009625287001, u'iteration': 0, u'remaining_time': 0.0182880453, u'learn': [u'inf', u'inf', 0.5]}, {u'test': [u'nan', u'nan', 1], u'passed_time': 0.001608976648, u'iteration': 19, u'remaining_time': 0.01448078983, u'learn': [u'nan', u'nan', 1]}]}, 'total_iterations': 20, 'passed_iterations': 19}, 'path': 'new_train_with_nan', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))

In [8]:
w = CatboostIpythonWidget('new_train_best')
w.update_widget()

CatboostIpythonWidget(data={'new_train_best': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': 1, u'name': u'CtrFactor'}, {u'best_value': u'Max', u'name': u'Accuracy'}, {u'best_value': u'Max', u'name': u'F1'}], u'learn_sets': [u'learn'], u'launch_mode': u'Train', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': 1, u'name': u'CtrFactor'}, {u'best_value': u'Max', u'name': u'Accuracy'}, {u'best_value': u'Max', u'name': u'F1'}], u'iteration_count': 100, u'test_sets': [u'test']}, 'iterations': [{u'test': [0.6607449728, 1.548304325, 0.7920792079, 0.8727272727], u'passed_time': 0.03734903517, u'iteration': 0, u'remaining_time': 3.697554482, u'learn': [0.6570221954, 1.513076028, 0.8712871287, 0.9230769231]}, {u'test': [0.5906788517, 1.429207717, 0.8514851485, 0.9122807018], u'passed_time': 0.09144038334, u'iteration': 3, u'remaining_time': 2.1945692, u'learn': [0.5765213693, 1.397275893, 0.9504950495, 0.9693251534]}, {u'test': [0.5294081398, 1.327218479, 0.801980198, 0.8780487805], u'passed_time': 0.1498329623, u'iteration': 6, u'remaining_time': 1.990637928, u'learn': [0.4965565457, 1.296913761, 0.9801980198, 0.9875]}, {u'test': [0.4923750015, 1.264317914, 0.8118811881, 0.8848484848], u'passed_time': 0.1800515089, u'iteration': 9, u'remaining_time': 1.62046358, u'learn': [0.4515523969, 1.23520323, 0.9603960396, 0.975308642]}, {u'test': [0.4596429304, 1.210149307, 0.8415841584, 0.9047619048], u'passed_time': 0.2234160652, u'iteration': 12, u'remaining_time': 1.495169052, u'learn': [0.4056997658, 1.183738659, 0.9603960396, 0.975308642]}, {u'test': [0.4342947367, 1.174199481, 0.8217821782, 0.8928571429], u'passed_time': 0.2669503018, u'iteration': 15, u'remaining_time': 1.401489084, u'learn': [0.3749026422, 1.150759822, 0.9504950495, 0.9693251534]}, {u'test': [0.4174339893, 1.146798789, 0.8118811881, 0.8862275449], u'passed_time': 0.3424343458, u'iteration': 18, u'remaining_time': 1.459851685, u'learn': [0.3411370878, 1.125533456, 0.9504950495, 0.9689440994]}, {u'test': [0.4009468918, 1.121379529, 0.8118811881, 0.8862275449], u'passed_time': 0.372739085, u'iteration': 21, u'remaining_time': 1.321529483, u'learn': [0.3149802077, 1.097170284, 0.9504950495, 0.9689440994]}, {u'test': [0.3902366245, 1.102247176, 0.8217821782, 0.8928571429], u'passed_time': 0.439056537, u'iteration': 24, u'remaining_time': 1.317169611, u'learn': [0.2982052544, 1.081790179, 0.9603960396, 0.975]}, {u'test': [0.3833275536, 1.091684308, 0.8316831683, 0.8982035928], u'passed_time': 0.4984792158, u'iteration': 27, u'remaining_time': 1.281803698, u'learn': [0.2800817558, 1.071575481, 0.9603960396, 0.975]}, {u'test': [0.3773738278, 1.08440227, 0.8217821782, 0.8928571429], u'passed_time': 0.5621775168, u'iteration': 30, u'remaining_time': 1.251298344, u'learn': [0.2670827411, 1.063740129, 0.9603960396, 0.975]}, {u'test': [0.3743189035, 1.074628502, 0.8316831683, 0.8982035928], u'passed_time': 0.6017725105, u'iteration': 33, u'remaining_time': 1.168146638, u'learn': [0.2589484264, 1.056478049, 0.9603960396, 0.975]}, {u'test': [0.3678527012, 1.065636366, 0.8217821782, 0.8928571429], u'passed_time': 0.643363916, u'iteration': 36, u'remaining_time': 1.095457479, u'learn': [0.249627837, 1.049199854, 0.9603960396, 0.975308642]}, {u'test': [0.3614593759, 1.058048889, 0.8217821782, 0.8928571429], u'passed_time': 0.6634086582, u'iteration': 39, u'remaining_time': 0.9951129873, u'learn': [0.2411296875, 1.042273274, 0.9504950495, 0.9689440994]}, {u'test': [0.3564756393, 1.052026857, 0.8217821782, 0.8928571429], u'passed_time': 0.7051454341, u'iteration': 42, u'remaining_time': 0.9347276685, u'learn': [0.228729877, 1.037145772, 0.9504950495, 0.9689440994]}, {u'test': [0.3525063436, 1.047082765, 0.8217821782, 0.8928571429], u'passed_time': 0.7259003504, u'iteration': 45, u'remaining_time': 0.8521438896, u'learn': [0.2231930231, 1.031812428, 0.9504950495, 0.96894409

In [9]:
w = CatboostIpythonWidget('new_eval')
w.update_widget()

CatboostIpythonWidget(data={'new_eval': {'content': {'data': {'meta': {u'name': u'', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': 1, u'name': u'CtrFactor'}, {u'best_value': u'Max', u'name': u'Accuracy'}, {u'best_value': u'Max', u'name': u'F1'}], u'learn_sets': [], u'launch_mode': u'Eval', u'learn_metrics': [], u'iteration_count': 21, u'test_sets': [u'eval_dataset']}, 'iterations': [{u'iteration': 0, u'eval_dataset': [0.6931471806, 1.564356436, 0.2178217822, 0]}, {u'iteration': 1, u'eval_dataset': [0.5456798805, 1.353746203, 0.9504950495, 0.9689440994]}, {u'iteration': 2, u'eval_dataset': [0.4511517436, 1.23699816, 0.9603960396, 0.9746835443]}, {u'iteration': 3, u'eval_dataset': [0.3845745186, 1.163696215, 0.9504950495, 0.9689440994]}, {u'iteration': 4, u'eval_dataset': [0.3345197242, 1.120394818, 0.9405940594, 0.9625]}, {u'iteration': 5, u'eval_dataset': [0.2979979584, 1.081403202, 0.9504950495, 0.9685534591]}, {u'iteration': 6, u'eval_dataset': [0.2709913454, 1.067632539, 0.9405940594, 0.9625]}, {u'iteration': 7, u'eval_dataset': [0.2538259936, 1.053445205, 0.9405940594, 0.9625]}, {u'iteration': 8, u'eval_dataset': [0.240742516, 1.043844644, 0.9504950495, 0.9689440994]}, {u'iteration': 9, u'eval_dataset': [0.2243983177, 1.034321685, 0.9504950495, 0.9689440994]}, {u'iteration': 10, u'eval_dataset': [0.2118324021, 1.02809414, 0.9504950495, 0.9689440994]}, {u'iteration': 11, u'eval_dataset': [0.2006788413, 1.024411774, 0.9603960396, 0.975]}, {u'iteration': 12, u'eval_dataset': [0.1874678551, 1.020728881, 0.9801980198, 0.9873417722]}, {u'iteration': 13, u'eval_dataset': [0.1803974807, 1.017450194, 0.9702970297, 0.9811320755]}, {u'iteration': 14, u'eval_dataset': [0.1731203764, 1.015343809, 0.9702970297, 0.9811320755]}, {u'iteration': 15, u'eval_dataset': [0.1654083437, 1.016043776, 0.9801980198, 0.9873417722]}, {u'iteration': 16, u'eval_dataset': [0.1558602922, 1.015133523, 0.9801980198, 0.9873417722]}, {u'iteration': 17, u'eval_dataset': [0.1461424435, 1.014093779, 0.9900990099, 0.9937106918]}, {u'iteration': 18, u'eval_dataset': [0.1409100552, 1.011831837, 0.9900990099, 0.9937106918]}, {u'iteration': 19, u'eval_dataset': [0.1364639812, 1.008830016, 0.9900990099, 0.9937106918]}, {u'iteration': 20, u'eval_dataset': [0.1329624159, 1.007691136, 0.9900990099, 0.9937106918]}]}, 'total_iterations': 21, 'passed_iterations': 20}, 'path': 'new_eval', 'name': 'current'}}, layout=Layout(align_self=u'stretch', height=u'500px'))

In [10]:
w = CatboostIpythonWidget('subdirs')
w.update_widget(subdirs = True)

CatboostIpythonWidget(data={'subdirs/new_train_multi': {'content': {'data': {'meta': {u'name': u'experiment', u'test_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': 0.5, u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'learn_sets': [u'learn'], u'launch_mode': u'Train', u'learn_metrics': [{u'best_value': u'Min', u'name': u'Logloss'}, {u'best_value': 0.5, u'name': u'RMSE'}, {u'best_value': u'Max', u'name': u'AUC'}], u'iteration_count': 20, u'test_sets': [u'test', u'test1']}, 'iterations': [{u'test1': [0.6600097926, 0.8186408447, 0.7882716049], u'iteration': 0, u'remaining_time': 0.2270625262, u'passed_time': 0.01195065927, u'learn': [0.6565050203, 0.8045194116, 0.9427502877], u'test': [0.6600097926, 0.8186408447, 0.7882716049]}, {u'test1': [0.632232761, 0.7421424448, 0.8009259259], u'iteration': 1, u'remaining_time': 0.1189260641, u'passed_time': 0.01321400713, u'learn': [0.6255581951, 0.7309581064, 0.9462025316], u'test': [0.632232761, 0.7521424448, 0.8009259259]}, {u'test1': [0.6095533415, 0.686701294, 0.8104938272], u'iteration': 2, u'remaining_time': 0.0990137329, u'passed_time': 0.01747301169, u'learn': [0.5972445019, 0.6674712405, 0.9571346375], u'test': [0.6095533415, 0.6986701294, 0.8104938272]}, {u'test1': [0.5900602047, 0.6405211024, 0.8703703704], u'iteration': 3, u'remaining_time': 0.07954403276, u'passed_time': 0.01988600819, u'learn': [0.576111189, 0.6163792864, 0.9833141542], u'test': [0.5900602047, 0.6505211024, 0.8703703704]}, {u'test1': [0.5665918765, 0.5881484265, 0.8734567901], u'iteration': 4, u'remaining_time': 0.06665792764, u'passed_time': 0.02221930921, u'learn': [0.5480418822, 0.551996609, 0.9907940161], u'test': [0.5665918765, 0.5981484265, 0.8734567901]}, {u'test1': [0.5510878118, 0.5575146714, 0.8666666667], u'iteration': 5, u'remaining_time': 0.05557157962, u'passed_time': 0.02381639127, u'learn': [0.530200324, 0.5112019609, 0.9838895282], u'test': [0.5510878118, 0.5675146714, 0.8666666667]}, {u'test1': [0.5371883975, 0.5358694973, 0.862345679], u'iteration': 6, u'remaining_time': 0.05124686866, u'passed_time': 0.02759446774, u'learn': [0.5093274822, 0.4693390275, 0.9815880322], u'test': [0.5371883975, 0.5458694973, 0.862345679]}, {u'test1': [0.5196093687, 0.5064592834, 0.8641975309], u'iteration': 7, u'remaining_time': 0.04389493786, u'passed_time': 0.02926329191, u'learn': [0.4892220617, 0.4324720121, 0.9867663982], u'test': [0.5196093687, 0.5164592834, 0.8641975309]}, {u'test1': [0.514718047, 0.4897642919, 0.8641975309], u'iteration': 8, u'remaining_time': 0.03674909852, u'passed_time': 0.03006744424, u'learn': [0.4848357164, 0.4135370112, 0.9867663982], u'test': [0.514718047, 0.4997642919, 0.8641975309]}, {u'test1': [0.5070544057, 0.4667807491, 0.8648148148], u'iteration': 9, u'remaining_time': 0.03104546668, u'passed_time': 0.03104546668, u'learn': [0.476894303, 0.391844182, 0.9884925201], u'test': [0.5070544057, 0.4767807491, 0.8648148148]}, {u'test1': [0.4884107139, 0.4556478835, 0.8691358025], u'iteration': 10, u'remaining_time': 0.02856870378, u'passed_time': 0.03491730462, u'learn': [0.4559441537, 0.368178344, 0.9942462601], u'test': [0.4884107139, 0.4656478835, 0.8691358025]}, {u'test1': [0.475222362, 0.4549231047, 0.8691358025], u'iteration': 11, u'remaining_time': 0.02581716555, u'passed_time': 0.03872574833, u'learn': [0.4395892343, 0.3601795584, 0.9902186421], u'test': [0.475222362, 0.4649231047, 0.8691358025]}, {u'test1': [0.4671390682, 0.4602490774, 0.8635802469], u'iteration': 12, u'remaining_time': 0.02137410484, u'passed_time': 0.03969476613, u'learn': [0.4299335594, 0.3634165349, 0.9856156502], u'test': [0.4671390682, 0.4702490774, 0.8635802469]}, {u'test1': [0.4593684635, 0.4540512426, 0.8709876543], u'iteration': 13, u'remaining_time': 0.01741411983, u'passed_time': 0.04063294628, u'learn': [0.4239420933, 0.3563966828, 0.9907940161], u'test': [0.4593684635, 0.4640512426, 0.8709876543]}, {u'test1': [0.4580933529, 0.4778625883, 0.854629629